In [2]:
from astropy.coordinates import SkyCoord, EarthLocation
import astropy.coordinates as coord
import astropy.units as u
from tabulate import tabulate
from astropy.time import Time
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
%load_ext autoreload
%autoreload 2
from Helper_funcs import *
from Scheduler import *

wind10toast not installed


[1, 2, 3, 4]


In [8]:
connect = sqlite3.connect('Database.db')
add_randome_data(10)

INSERT INTO Observations(PID, Name, EMail, Phone, Completed_by, Submission_Date, Observer_type, time_sensitive, obsIDs, missing_obsIDs, total_length, logsheet, Obs_days) VALUES('1.0', 'User', 'User@kapteyn.nl', '001123121', '15-07-2021', '17-07-2020', 'OA', 'True', '[1, 2]', '[1, 2]', '300.0', 'None', 'None');
INSERT INTO Observations(PID, Name, EMail, Phone, Completed_by, Submission_Date, Observer_type, time_sensitive, obsIDs, missing_obsIDs, total_length, logsheet, Obs_days) VALUES('2.0', 'User', 'User@kapteyn.nl', '001123121', '22-09-2021', '14-01-2021', 'Public', 'True', '[3, 4, 5, 6]', '[3, 4, 5, 6]', '720.0', 'None', 'None');
INSERT INTO Observations(PID, Name, EMail, Phone, Completed_by, Submission_Date, Observer_type, time_sensitive, obsIDs, missing_obsIDs, total_length, logsheet, Obs_days) VALUES('3.0', 'User', 'User@kapteyn.nl', '001123121', '12-09-2021', '16-03-2020', 'Outreach/schools', 'True', '[7, 8]', '[7, 8]', '240.0', 'None', 'None');
INSERT INTO Observations(PID, Name

In [3]:
connect = sqlite3.connect('Database.db')
sqlite_get_tables(connect)

['sorted', 'Observations', 'Schedule']

In [4]:
cont = sqlite_retrieve_table(connect, 'Observations')
for i in cont:
    print(i)

{'PID': 1, 'Name': 'User', 'EMail': 'User@kapteyn.nl', 'Phone': '001123121', 'Completed_by': '15-07-2021', 'Submission_Date': '17-07-2020', 'Observer_type': 'OA', 'time_sensitive': 'True', 'obsIDs': ['1', ' '], 'missing_obsIDs': ['1', ' '], 'total_length': 300, 'logsheet': None, 'Obs_days': None}
{'PID': 2, 'Name': 'User', 'EMail': 'User@kapteyn.nl', 'Phone': '001123121', 'Completed_by': '22-09-2021', 'Submission_Date': '14-01-2021', 'Observer_type': 'Public', 'time_sensitive': 'True', 'obsIDs': ['3', ' 4', ' 5', ' '], 'missing_obsIDs': ['3', ' 4', ' 5', ' '], 'total_length': 720, 'logsheet': None, 'Obs_days': None}
{'PID': 3, 'Name': 'User', 'EMail': 'User@kapteyn.nl', 'Phone': '001123121', 'Completed_by': '12-09-2021', 'Submission_Date': '16-03-2020', 'Observer_type': 'Outreach/schools', 'time_sensitive': 'True', 'obsIDs': ['7', ' '], 'missing_obsIDs': ['7', ' '], 'total_length': 240, 'logsheet': None, 'Obs_days': None}
{'PID': 4, 'Name': 'User', 'EMail': 'User@kapteyn.nl', 'Phone': 

In [ ]:
from astroplan import Constraint
from scipy.interpolate import NearestNDInterpolator, interp1d
import numpy as np





# Trials of Scheduler

In [2]:
%load_ext autoreload
%autoreload 2
from Scheduler import Scheduling

wind10toast not installed


In [9]:
file_path = r'.'
config = ''
sel=Scheduling(file_path, config)

Image directory for todays run already existed!


In [10]:
sel.schedule

Schedule containing 0 observing blocks between 2021-08-21 18:51:09.000 and 2021-08-22 04:27:23.000

In [25]:
import astropy.units as u
type(sel.schedule.open_slots[0].duration.to_value(u.second))

numpy.float64

In [29]:
os.listdir('.')
#(dt.date.today()-dt.timedelta(days=1)).strftime('%Y%m%d')

['.ipynb_checkpoints',
 'README.md',
 '20210821',
 'trials.ipynb',
 'Background_Script.py',
 'autohotkey_triouts.ahk',
 'LICENSE',
 'trial_plan.txt',
 'Helper_funcs.py',
 'openweatherapi.ipynb',
 'Database.db',
 'database_handling.py',
 '__pycache__',
 'Scheduler.py',
 'autohotkey_triouts.exe',
 '.git',
 'config.ini',
 'data_transfer.py',
 'sky_bright']

In [11]:
sel.priority_ordered_list

[{'obsID': 12,
  'object': 'M 73',
  'PID': 5,
  'Filter': 'G*',
  'exposure': 60,
  'binning': 1,
  'airmass': None,
  'moon': 0,
  'seeing': None,
  'sky_brightness': None,
  'Observer_type': '12',
  'time_sensitive': 'False',
  'Submission_Date': '23-06-2020',
  'Completed_by': '21-09-2021',
  'total_length': 60,
  'Rarity': 1,
  'number_of_exposures': 2,
  'priority': 6.8},
 {'obsID': 23,
  'object': 'M 74',
  'PID': 10,
  'Filter': 'None',
  'exposure': 120,
  'binning': 1,
  'airmass': 1,
  'moon': 0,
  'seeing': None,
  'sky_brightness': None,
  'Observer_type': '23',
  'time_sensitive': 'True',
  'Submission_Date': '17-12-2020',
  'Completed_by': '22-10-2021',
  'total_length': 120,
  'Rarity': 1,
  'number_of_exposures': 4,
  'priority': 6.7},
 {'obsID': 1,
  'object': 'M 41',
  'PID': 1,
  'Filter': 'B*',
  'exposure': 240,
  'binning': 1,
  'airmass': 2,
  'moon': 0,
  'seeing': None,
  'sky_brightness': None,
  'Observer_type': '1',
  'time_sensitive': 'True',
  'Submission

# Seeing

In [25]:
import requests
import pandas as pd

url = 'https://www.meteoblue.com/en/weather/outdoorsports/seeing/lauwersoog_netherlands_2751849'

html = requests.get(url).content


In [26]:
html=html.split(b'\n')

In [27]:
for i in range(len(html)):
    if b'<table class="table-seeing">' in html[i]:
        break
for j in range(i,len(html)):
    if b'</tbody>' in html[j]:
        break
html = html[i:j]

In [5]:
for i in range(len(html)):
    if b'/thead' in html[i]:
        break
headers = html[:i]

In [6]:
data = html[i:]

In [13]:
rows = []
cond = False
ls = []
for i in range(len(data)):
    if b'<tr class="hour-row night"' in data[i]:
        cond = True
    elif b'</tr>' in data[i]:
        cond = False
        rows.append(ls)
        ls = []
    elif cond:
        ls.append(data[i])


In [21]:
arr = []
for i in range(len(rows[1])):
    if b'</td>' in rows[1][i]:
        val =rows[1][i].split(b'<')[0].split(b'>')
        if len(val) >

In [22]:
arr

[[b'                    ', b'2'],
 [b'                        0                    '],
 [b'                        0                    '],
 [b'                        0                    '],
 [b'                                        '],
 [b'                        4                    '],
 [b'                        3                    '],
 [b'                    '],
 [b'                        02.1                    '],
 [b'                        03.8                    '],
 [b'                    '],
 [b'                        62&thinsp;\xc2\xb0F                    '],
 [b'                    '],
 [b'                                    ']]